# Process ROH Results into one big dataframe
Contains cleaning lines (i.e. to remove duplicates)

In [ ]:
import numpy as np
import os  # For Saving to Folder
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colorbar as clb
import matplotlib.colors as cls
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.basemap import Basemap
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

import socket
import os as os
import sys as sys
import multiprocessing as mp

socket_name = socket.gethostname()
print(socket_name)
if socket_name == "VioletQueen":
    path = "/home/harald/git/HAPSBURG/"   # The Path on Harald's machine
elif socket_name.startswith("midway2"):
    print("Midway jnovmbre partition detected.")
    path = "/project2/jnovembre/hringbauer/HAPSBURG/"  # The Path on Midway Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)
print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(f"CPU Count: {mp.cpu_count()}")

### Additional Imports from Support Packages
sys.path.append("./PackagesSupport/")
from pp_individual_roh_csvs import extract_sub_df_geo_kw

### Functions that pre-process Data
Add "region" Field. Add "color" (based on Time) field

In [6]:
def pre_process_roman_df(df, age_error=0, remove_sard=False):
    """Preprocess and return roman df and adds colors"""
    color_dict = {"Medieval/EarlyModern":"yellow", "Imperial":"red", "Iron/Republic":"magenta", 
                  "LateAntiquity":"orange", "Copper Age":"aquamarine", "Neolithic":"dodgerblue", 
                  "Mesolithic":"purple", "(not included in analyses)":"gray"}
    df["color"] = df["clst"].map(color_dict)
    if age_error>0:
        df["age"]+= np.random.random(len(df))*age_error - age_error/2
    
    df["region"]="Rome" 
    ### Modify Sardinians
    idx_sar = (df["clst"] == "(not included in analyses)")
    df.loc[idx_sar,"region"] = "Sardinia"
    return df

def pre_process_iberia_df(df, age_error=0):
    """Preprocess and return roman df and adds colors"""
    df["color"]="silver"

    ### WHG Coloring
    hg_terms = ["HG", "Meso", "ElMiron"]
    idx = df["clst"].str.contains('|'.join(hg_terms))
    df.loc[idx, "color"]="purple"
    df.loc[idx, "clst"]="Mesolithic"
    
    ### EN Coloring
    en_terms = ["Iberia_EN"]
    idx = df["clst"].str.contains('|'.join(en_terms))
    df.loc[idx,"color"]="blue"
    df.loc[idx,"clst"]="Early Neolithic"
    
    ### Middle Late Neoltihic
    mn_terms = ["MN", "MLN", "MN", "LN"]
    idx = df["clst"].str.contains('|'.join(mn_terms))
    df.loc[idx,"color"]="lightblue"
    df.loc[idx,"clst"]="Middle/Late Neolithic"
    
    ### Muslim Burials
    en_terms = ["SE_Iberia_c.10-16CE"]
    idx = df["clst"].str.contains('|'.join(en_terms))
    df.loc[idx,"color"]="red"
    df.loc[idx,"clst"]="Muslim Period"
        
    if age_error>0:
        df["age"]+= np.random.random(len(df)) * age_error - age_error/2      
    return df

def pre_process_reich_df(df, age_error=0, del_strings=[]):
    """Preprocess and return roman df and adds colors.
    del_strings: iid column in df that contains this list of strings
    gets deleted"""
    ### Fix Geography
    df.loc[df["iid"]=="I7554", "lon"] = -3.249  # Flip Wrong Latitude Atlantic
    df.loc[df["iid"]=="Aconcagua.SG", "lat"] = -32.65  # Flip Wrong Latitude (32.64 is in Atlantic)
    
    ### Delete individuals
    for ds in del_strings:
        df = df[~df["iid"].str.contains(ds)]
    
    ### WHG Coloring
    hg_terms = ["HG", "Meso", "ElMiron", "Iron Gates", "Loschbour"]
    idx = ((df["clst"].str.contains('|'.join(hg_terms))) | (df["age"]>10500)) & (df["age"]>5000)
    df.loc[idx,"color"]="purple"
    df.loc[idx,"clst"]="Mesolithic"
    
    ### EN Coloring
    en_terms = ["EN", "Early Neol", "Neolithic", "Cardial", "MN", "LN", "MLN", "Ukraine_N", "Peloponnese_N"]
    idx = df["clst"].str.contains('|'.join(en_terms)) & (df["age"]>5500)
    df.loc[idx,"color"] = "aqua"
    df.loc[idx,"clst"] = "Neolithic"
    
    ### Antatolia Farmers
    en_terms = ["Anatolia_N", "Anatolia Farmers"]
    idx = df["clst"].str.contains('|'.join(en_terms))
    df.loc[idx,"color"]="blue"
    df.loc[idx,"clst"]="Anatolia Farmers"
    
    en_terms = ["Canaanite"]
    idx = df["clst"].str.contains('|'.join(en_terms))
    df.loc[idx,"color"]="red"
    df.loc[idx,"clst"]="Canaanite"
    
    en_terms = ["Sar-Nur"]
    idx = df["clst"].str.contains('|'.join(en_terms))
    df.loc[idx,"color"]="red"
    df.loc[idx,"clst"]="Nuragic"
    
    en_terms = ["skythian", "Skythian"]
    idx = df["clst"].str.contains('|'.join(en_terms))
    df.loc[idx,"color"]="orange"
    df.loc[idx,"clst"]="Skythian"
    
    if age_error>0:
        df["age"]+= np.random.random(len(df)) * age_error - age_error/2
    return df

############################################################################
### Post-Process Regions
def set_regions_from_csv(csv_path, df, output=True, sep=","):
    """Set Region coumn in df, by loading coordinates from csv_path"""
    df_regions= pd.read_csv(csv_path, sep=sep)
    for index, row in df_regions.iterrows():
        region = row["Region"] 
        if output:
            print(f"Doing {region}...")
        kw = str(row["Keywords"]).split("|") # produce list from Keywords
        df_t = extract_sub_df_geo_kw(df_all, row["Lat_low"], row["Lat_high"], row["Lon_low"], 
                                     row["Lon_high"], kw, output=output)
        idx = df["iid"].isin(df_t["iid"]) # Get Indices of Sub Dataframe
        df.loc[idx, "region"] = row["Region"] 
    return df

# Goal:
Have one Master Dataframe. With Region field

In [3]:
### Roman Dataframe
df_rome = pd.read_csv("./Empirical/1240k/Antonio/combined_roh05.csv", sep="\t")
df_rome = pre_process_roman_df(df_rome, age_error=0, remove_sard=False)
df_rome.drop(columns='age_range', inplace=True)
cols = df_rome.columns # Remember the Column Names

### Reich Dataframe
# Define Individuals we want to delete (Duplicates/Neanderthals)
del_strings = ["Loschbour_snpAD.DG", "Mezmaiskaya", "Ishim_published.DG", "Vindija_snpAD", 
               "Kostenki14.SG", "Goyet", "Spy", "Denisova", "Altai", "Les_Cottes", "Anzick.SG",
               "Russia_Karelia_HG.SG", "I0001", "I2966_all", "I5259_all", "I4450_all",
               "I4105_all", "I4106_all", "I3921_all"]
df_r = pd.read_csv("./Empirical/Eigenstrat/Reichall/combined_roh05.csv", sep="\t")
df_r = pre_process_iberia_df(df_r, age_error=0)
df_r = pre_process_reich_df(df_r, del_strings=del_strings)
df_r['region'] = "all"   ### Modify this

### Sardinians from Marcus et all
df_sard = pd.read_csv("./Empirical/1240k/MarcusAncs/combined_roh05.csv", sep="\t")
df_sard = pre_process_reich_df(df_sard)
df_sard["region"]="Sardinia"
df_sard = df_sard[cols]

### Iberia from Olalde19
df_ib = pd.read_csv("./Empirical/Eigenstrat/Olalde19/combined_roh05.csv", sep="\t")
df_ib = pre_process_iberia_df(df_ib, age_error=0)
df_ib["region"]="Iberia"
df_ib.drop(columns='age_range', inplace=True)

In [92]:
### Human Origin Data

### Concatenate all the Dataframes

In [4]:
df_all = pd.concat([df_rome, df_r, df_sard, df_ib])
df_all = pre_process_reich_df(df_all)

### Set the Regions

In [7]:
csv_path = "./Data/RegionDefinition/regions.csv"
df_t = set_regions_from_csv(csv_path, df_all)
#df_t = pre_process_iberia_df(df_t, age_error=0)  # Hack for having Iberians right

Doing Iberia...
Found 159 Individuals; 143 from Geography
Doing Balkans...
Found 291 Individuals; 167 from Geography
Doing Aegan...
Found 81 Individuals; 73 from Geography
Doing Central Europe...
Found 244 Individuals; 244 from Geography
Doing Black Sea...
Found 49 Individuals; 49 from Geography
Doing North Africa...
Found 5 Individuals; 4 from Geography
Doing Britain...
Found 240 Individuals; 223 from Geography
Doing Baltic Sea...
Found 74 Individuals; 74 from Geography
Doing Sardinia...
Found 40 Individuals; 40 from Geography
Doing Levante...
Found 27 Individuals; 26 from Geography
Doing Vanuatu...
Found 19 Individuals; 19 from Geography
Doing Steppe...
Found 229 Individuals; 229 from Geography
Doing Patagonia...
Found 8 Individuals; 8 from Geography
Doing Andean...
Found 25 Individuals; 25 from Geography
Doing Pacific NW...
Found 16 Individuals; 16 from Geography
Doing Atlantic Coast...
Found 15 Individuals; 15 from Geography


### Save the Summary Dataframe

In [9]:
savepath="./Empirical/roh_all_inds.csv"
if len(savepath)>0:
    df_all.to_csv(savepath, sep="\t", index=False)
    print(f"Saved {len(df_all)} Individual ROH to: {savepath}")

Saved 1855 Individual ROH to: ./Empirical/roh_all_inds.csv


# Area 51

In [51]:
df_region = pd.read_csv(csv_path, sep=',')

In [52]:
df_region

,Region,Lat_low,Lat_high,Lon_low,Lon_high,Keywords
0,Iberia,35.95,44.0,-10.00,4.0,Iberia|Portugal|Spain
1,Balkans,42.20,46.9,13.05,23.9,Balkans|Serbia|Hungary
2,Aegan,36.30,41.0,18.00,40.4,Anatolia|Greece
3,Central Europe,45.00,52.0,5.00,16.9,Austria|Switzerland
4,Black Sea,44.00,55.0,25.00,39.5,Ukraine
5,North Africa,26.00,36.0,-11.00,25.0,Morocco
6,Britain,49.00,60.0,-12.00,3.0,Britain|Scot|Wales|England|Orkney
7,Baltic Sea,53.00,67.0,7.00,35.0,Baltic
8,Sardinia,38.70,41.3,8.00,9.9,Sar-
9,Levante,30.00,37.0,32.00,38.0,Levant|Israel|Canaanite


In [ ]:
df_r[df_r.duplicated(subset=["lat", "lon", "age"], keep=False)].sort_values(by="age", ascending=False)